In [4]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np

In [2]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 单一的单元和一个sigmoid激活结束模型，因为是二分模型。使用binary_crossentropy loss。
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

/Users/imperatore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  
/Users/imperatore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/Users/imperatore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
            '../../../../assets/dogvscat/train/',  # this is the target directory
            target_size=(150, 150),  # all images will be resized to 150x150
            batch_size=32,
            class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
    
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
            '../../../../assets/dogvscat/val/',
            target_size=(150, 150),
            batch_size=32,
            class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [6]:
model.fit_generator(
            train_generator,
            samples_per_epoch=2000,
            nb_epoch=10,
            validation_data=validation_generator,
            nb_val_samples=800)
model.save_weights('dvc_0.h5')  # always save your weights after training or during training

/Users/imperatore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/Users/imperatore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=62, epochs=10, validation_steps=800)`
  


Epoch 1/10
62/62 [==============================] - 440s 7s/step - loss: 0.8039 - acc: 0.4940 - val_loss: 0.6847 - val_acc: 0.4999
Epoch 2/10
62/62 [==============================] - 429s 7s/step - loss: 0.7042 - acc: 0.5418 - val_loss: 0.6668 - val_acc: 0.5980
Epoch 3/10
62/62 [==============================] - 349s 6s/step - loss: 0.6895 - acc: 0.5771 - val_loss: 0.6503 - val_acc: 0.6370
Epoch 4/10
62/62 [==============================] - 363s 6s/step - loss: 0.6740 - acc: 0.5872 - val_loss: 0.6346 - val_acc: 0.6255
Epoch 5/10
62/62 [==============================] - 350s 6s/step - loss: 0.6437 - acc: 0.6452 - val_loss: 0.6249 - val_acc: 0.6414
Epoch 6/10
62/62 [==============================] - 344s 6s/step - loss: 0.6335 - acc: 0.6452 - val_loss: 0.6544 - val_acc: 0.6035
Epoch 7/10
62/62 [==============================] - 337s 5s/step - loss: 0.6114 - acc: 0.6779 - val_loss: 0.6167 - val_acc: 0.6831
Epoch 8/10
62/62 [==============================] - 324s 5s/step - loss: 0.5992 - a

# VGG16 bottleneck

In [2]:
from keras.applications.vgg16 import VGG16  
model = VGG16(weights='imagenet', include_top=False) 

/Users/imperatore/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [92]:
model.save_weights('VGG16_WEIGHTS.h5')

In [5]:
datagen = ImageDataGenerator(
            rotation_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

In [6]:
train_generator = datagen.flow_from_directory(
            '../../../../assets/dogvscat/train/',
            target_size=(150, 150),
            batch_size=32,
            shuffle=False)

Found 24000 images belonging to 2 classes.


In [11]:
bottleneck_features_train = model.predict_generator(train_generator, len(train_generator))

In [100]:
# np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
# np.save('bottleneck_features_train.npy', bottleneck_features_train)

In [8]:
val_generator = datagen.flow_from_directory(
            '../../../../assets/dogvscat/val/',
            target_size=(150, 150),
            batch_size=32,
            shuffle=False)

Found 992 images belonging to 2 classes.


In [12]:
bottleneck_features_validation = model.predict_generator(val_generator, len(val_generator))

In [103]:
# np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
# np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

In [13]:
# train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
# train_data = np.load('bottleneck_features_train.npy')

# the features were saved in order, so recreating the labels is easy
# train_labels = np.array([0] * 1000 + [1] * 1000)
train_labels = train_generator.classes
# print(train_labels.shape)

# validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
# validation_data = np.load('bottleneck_features_validation.npy')

# validation_labels = np.array([0] * 400 + [1] * 400)
validation_labels = val_generator.classes

In [16]:
md = Sequential()
md.add(Flatten(input_shape=bottleneck_features_train.shape[1:]))
md.add(Dense(256, activation='relu', input_shape=bottleneck_features_train.shape[1:]))
md.add(Dropout(0.5))
md.add(Dense(1, activation='sigmoid'))
    
md.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

md.fit(bottleneck_features_train, train_labels,
          nb_epoch=50, batch_size=32,
          validation_data=(bottleneck_features_validation, validation_labels))
md.save_weights('bottleneck_fc_model.h5')

/Users/imperatore/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 24000 samples, validate on 992 samples
Epoch 1/50
24000/24000 [==============================] - 23s 947us/step - loss: 2.0748 - acc: 0.8644 - val_loss: 1.7393 - val_acc: 0.8901
Epoch 2/50
24000/24000 [==============================] - 18s 770us/step - loss: 1.5807 - acc: 0.8981 - val_loss: 2.1922 - val_acc: 0.8609
Epoch 3/50
24000/24000 [==============================] - 18s 750us/step - loss: 1.5061 - acc: 0.9039 - val_loss: 1.5893 - val_acc: 0.9002
Epoch 4/50
24000/24000 [==============================] - 17s 693us/step - loss: 1.4334 - acc: 0.9085 - val_loss: 1.4099 - val_acc: 0.9103
Epoch 5/50
24000/24000 [==============================] - 16s 685us/step - loss: 1.3682 - acc: 0.9132 - val_loss: 1.3555 - val_acc: 0.9103
Epoch 6/50
24000/24000 [==============================] - 17s 692us/step - loss: 1.3876 - acc: 0.9118 - val_loss: 1.5640 - val_acc: 0.9002
Epoch 7/50
24000/24000 [==============================] - 17s 698us/step - loss: 1.3059 - acc: 0.9165 - val_loss: 1.43

In [21]:
bottleneck_features_train.shape
md.predict(bottleneck_features_train[18000:18001])

array([[1.]], dtype=float32)